In [93]:
from rasa_nlu.model import Interpreter
import json

In [94]:
# nlu_config = """
# language: fr
# pipeline: tensorflow_embedding
# """

nlu_config = """
language: fr
pipeline:
- name: "nlp_spacy"
- name: "tokenizer_spacy"
- name: "intent_featurizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"
"""

# nlu_config = """
# language: fr
# pipeline: "spacy_sklearn"
# """

%store nlu_config > nlu_config.yml

Writing 'nlu_config' (str) to file 'nlu_config.yml'.


In [105]:
nlu_md = """
## intent:hello
- salut
- bonjour
- ça va
- hey
- hello
- coucou

## intent:search_dataset
- statistiques de [la ville de paris](organization)
- je cherche [les données](object) du [ministère de l'intérieur](organization)
- j'aimerai connaitre la [qualité de l'air](topic) à Paris
- Je voudrai avoir [les chiffres](object) de la [population française](topic)
- quel est [mon numéro](object) [RNA](topic) ?
- poule 2010
- je veux des données
- [vigicrue](topic) 2018
- base [SIRENE](topic)

## intent:thankyou
- merci
- c'est chouette
- cool !
- super !

## intent:bye
- au revoir
- à plus
- bye
- ciao
- bonne nuit

## lookup:organization
organisations.txt

## lookup:dataset
datasets.txt

## regex:year
- [0-9]{4}
"""
%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


## importation des noms d'organisations

In [47]:
import pandas as pd

In [51]:
df1 = pd.read_csv('../pilotage/data/organizations.csv', sep=";")

In [92]:
names = [ x for x in list(df['name'].astype(str).values) if x != 'nan' ]
accronyms = [ x for x in list(df['acronym'].astype(str).values) if x != 'nan' ]

txt = "\n".join(names + accronyms)
%store txt > organisations.txt

Writing 'txt' (str) to file 'organisations.txt'.


In [99]:
df2 = pd.read_csv('../pilotage/data/datasets.csv', sep=";")

/home/tk/.virtualenvs/notebooks/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
df2.head()

names = [ x for x in list(df2['title'].astype(str).values) if x != 'nan' ]
accronyms = [ x for x in list(df2['acronym'].astype(str).values) if x != 'nan' ]

txt = "\n".join(names + accronyms)
%store txt > datasets.txt

Writing 'txt' (str) to file 'datasets.txt'.


## Entrainement

In [108]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/tk/.virtualenvs/notebooks/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/tk/.virtualenvs/notebooks/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/tk/.virtualenvs/notebooks/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/tk/.virtualenvs/notebooks/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: Und

In [103]:
interpreter = Interpreter.load("./models/current/nlu")

In [104]:
message = "Pays de Morlaix 2017"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

{
  "intent": {
    "name": "search_dataset",
    "confidence": 0.5247645051737571
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "search_dataset",
      "confidence": 0.5247645051737571
    },
    {
      "name": "thankyou",
      "confidence": 0.17211134414350557
    },
    {
      "name": "bye",
      "confidence": 0.1598025648206477
    },
    {
      "name": "hello",
      "confidence": 0.14332158586208962
    }
  ],
  "text": "Pays de Morlaix 2017"
}
